In [70]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
from scipy import stats, optimize
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [71]:
de_df = pd.read_csv('adult_diff_exp.csv', index_col=0)

de_df.head()

,wt+20°C_vs_nhl-2+20°C_PRJNA311537,wt+25°C_vs_nhl-2+25°C_PRJNA311537,ev_vs_NaCl_PRJNA421159,wt(EV)_vs_dpy(e88);skn-1(RNAi)_PRJNA421159,wt(EV)_vs_dpy(e88)_PRJNA421159,ev_vs_skn-1(RNAi)_PRJNA421159,wt_vs_morc-1(tm6048)_PRJNA353647,wt+20°C_vs_nhl-2+20°C_PRJNA398116,wt+25°C_vs_nhl-2+25°C_PRJNA398116,wt_vs_cey-1(rrr12);cey-4(ok585)_PRJNA265852,...,fed+20ºC_vs_starved+20ºC_PRJEB14837,no treatment_vs_DMSO_PRJNA391939,no treatment_vs_AZT_PRJNA391939,no treatment_vs_d4T_PRJNA391939,no treatment_vs_FLT_PRJNA391939,wt_vs_isp-1;sod-3_PRJNA376493,wt_vs_isp-1_PRJNA376493,wt_vs_emr-1(gk119)_PRJNA190895,wt_vs_lem-2(tm1582)_PRJNA190895,wt_vs_lem-2(tm1582);emr-1(RNAi)_PRJNA190895
2L52.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2RSSE.1,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4R79.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6R55.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aagr-1,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# compute Hamming distance matrix for samples

upreg_df = (de_df == 1).astype(int)
not_upreg_df = np.logical_not(upreg_df).astype(int)
downreg_df = (de_df == -1).astype(int)
not_downreg_df = np.logical_not(downreg_df).astype(int)
noreg_df = (de_df == 0).astype(int)
not_noreg_df = np.logical_not(noreg_df).astype(int)

D = np.dot(upreg_df.T, not_upreg_df) + np.dot(downreg_df.T, not_downreg_df) + np.dot(noreg_df.T, not_noreg_df)
ham_df = pd.DataFrame(D, index=de_df.columns, columns=de_df.columns)

In [73]:
# compute distance to null perturbation = num of differentially expressed genes
npd_df = np.sum(de_df != 0, axis=1)

npd_df

2L52.1      7
2RSSE.1     4
4R79.2      4
6R55.2      2
aagr-1     29
aagr-2     21
aagr-3     10
aagr-4     21
aak-1      16
aak-2      12
aakb-1     12
aakb-2     10
aakg-1     15
aakg-2     13
aakg-3      6
aakg-4     21
aakg-5     15
aap-1      11
aars-1     15
aars-2     20
aat-1      23
aat-2       9
aat-3       9
aat-4      14
aat-5      10
aat-6       9
aat-7       4
aat-8       1
aat-9      12
abce-1     25
           ..
ztf-11     16
ztf-13     17
ztf-14      7
ztf-15     16
ztf-16      8
ztf-17     13
ztf-18     11
ztf-2       4
ztf-20      7
ztf-22      7
ztf-23     13
ztf-25     19
ztf-26      7
ztf-27     15
ztf-28     18
ztf-29     12
ztf-3      10
ztf-30     16
ztf-4      17
ztf-6      16
ztf-7       7
ztf-8      12
ztf-9       8
zwl-1      17
zyg-1      15
zyg-11     22
zyg-12     16
zyg-8      11
zyg-9      15
zyx-1      13
Length: 20277, dtype: int64

In [112]:
# Require x = # of samples

def energy(x):
    n = len(ham_df.columns)
    combins = np.array(list(itertools.combinations(range(n), 2)))
    
    sq_err_f = lambda row: (abs(x[row[0]] - x[row[-1]]) - ham_df.iloc[row[0], row[-1]])**2
    sq_err = np.apply_along_axis(sq_err_f, axis=1, arr=combins)
    
    null_dist_f = lambda i: (abs(x[i]) - npd_df.iloc[i])**2
    null_dist_vf = np.vectorize(null_dist_f)
    null_dist = null_dist_vf(np.arange(0, n, 1))
    
    return np.sum(sq_err) + np.sum(null_dist)

115815789561

In [ ]:
optimize.basinhopping(energy, [0 for _ in range(len(ham_df.columns))], niter=1)